In [1]:
import pandas as pd
import json
import csv
import cplex
import time

In [12]:
def listar(x):
    rawData = []
    start = 0
    for i in range(len(x)):
        if x[i] == ' ' or x[i] == '\n':
            rawData.append(x[start:i])
            start = i + 1
    return rawData

def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end=" ")
        print()
        
def ler_grafo(u):
    arquive = open(u, 'r')
    content = arquive.read()
    rawData = []
    rawData = listar(content)

    vertices, arestas = [int(v) for v in rawData[:2]]
    edgeData = [json.loads(edge) for edge in rawData[2:]]
    print(str(vertices) + ' vertices, ' + str(arestas) + ' arestas, ')
    # print(edgeData)
    m1, m2 = [], []
    for i in range(vertices):
        m1.append([0]*vertices)
        m2.append([])
    
    for i in range(0,len(edgeData)-1, 2):
        m1[edgeData[i]][edgeData[i+1]] = 1
        m1[edgeData[i+1]][edgeData[i]] = 1

    if len(m1) != 0:
        for i in  range(len(m1)):
            for j in range(len(m1[i])):
                if m1[i][j] == 1:
                    m2[i].append(j+1)
    # print_matrix(m1)
    # print()
    # print_matrix(m2)
    return m2,vertices,arestas



In [13]:
def resolve_modelo(m2,vertices,arestas,k, timethresh):
    
    cpx = cplex.Cplex()

    y = cpx.variables.add(obj=[0] * vertices,
                             lb=[0] * vertices, ub=[1] * vertices,
                             types=['B'] * vertices,
                             names=['y_%d' % (p) for p in range(1,vertices+1)])
    h = cpx.variables.add(obj=[1] * vertices,
                            lb=[0] * vertices, ub=[1] * vertices,
                            types=['B'] * vertices,
                            names=['h_%d' % (z) for z in range(1,vertices+1)])

    #Restrição 2
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([y[i] for i in range(vertices)], [1.0]*vertices)],
        senses=['E'],
        rhs= [k])


    #Restrição 3
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]], [1.0,-1.0])
                  for i in range(vertices)],
        senses=['L'] * vertices,
        rhs=[0.0] * vertices)
    
    
    #Restrição 4
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[j-1]], [1.0,-1.0])
                  for i in range(vertices) for j in m2[i]],
        senses=['L'] * (arestas*2),
        rhs=[0.0] * (arestas*2))
    
    
    #Restrição 5
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]]+[y[j-1] for j in m2[i]], [1.0,-1.0]+[-1.0]*len(m2[i]))
                  for i in range(vertices)],
        senses=['G'] * vertices,
        rhs=[-len(m2[i]) for i in range(vertices)])

    
    cpx.parameters.threads.set(1)
    cpx.parameters.timelimit.set(timethresh)
    cpx.objective.set_sense(cpx.objective.sense.maximize)
    
    cpx.write('model.lp')
    start_time = time.time()
    cpx.solve()
    my_time = time.time() - start_time
    status = cpx.solution.get_status()
    # if status != 101:
    #     relaxation = [cpx.solution.MIP.get_best_objective(),
    #                     cpx.solution.MIP.get_cutoff,
    #                     # cpx.solution.MIP.get_incubent_node(),
    #                     # cpx.solution.MIP.get_mip_relative_gap(),
    #                     cpx.solution.MIP.get_num_cuts(cpx.solution.MIP.cut_type.cover),
    #                     cpx.solution.MIP.get_subproblem_status()]
    # else:
    relaxation = []

    return status, cpx.solution.get_objective_value(), cpx.solution.MIP.get_best_objective(), ','.join(relaxation), cpx.solution.get_objective_value(), my_time


In [14]:
# index = 0
# results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
#                                  'extra_params', 'result_ticks', 'result_time'])
# deuruim = {
#     'BA': False,
#     'bipartite': False,
#     'lollipop': False,
#     'star': False,
#     'cubic': False
# }
# time_threshold = 900
# for log_n in range(2,15):
#     n = 2**log_n
#     for graphClass in ["star", "cubic", "bipartite", "BA", "lollipop"]:
#         for k in range(1,4):
#             k = 2**(log_n + k - 4)
#             filename = f'inputs/{graphClass}/{graphClass}_{n}.txt'
#             print(f'Começando {filename}_k:{k}.------------------------>')
#             if not deuruim[graphClass]:
#                 m2,v,e = ler_grafo(filename)
#                 status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
#                 if status != 101:
#                     deuruim[graphClass] = True
#                 results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
#                 print(f'------------------------>{filename}_k:{k} feito.')

#                 index += 1
#             results.to_csv(f'./results4.csv')
# print("Fim.")


In [15]:
# index = 0
# results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
#                                  'extra_params', 'result_ticks', 'result_time'])
# deuruim = 0
# time_threshold = 2800
# if deuruim < 4:
#     for log_n in range(2,15):
#         n = 2**log_n
#         kStep = int(n/20)
#         if kStep == 0:
#             kStep = 1
#         for k in range(0,n,kStep):
#             filename = f'inputs/BA/BA_{n}.txt'
#             print(f'Começando {filename}_k:{k}.------------------------>')
#             m2,v,e = ler_grafo(filename)
#             status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
#             if status != 101:
#                 deuruim += 1
#             results.loc[index] = [f'{filename}_{k}', "BA", n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
#             print(f'------------------------>{filename}_k:{k} feito.')
#             index += 1
#             results.to_csv(f'./results5.csv')
# print("Fim.")


In [16]:
# index = 0
# results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
#                                  'extra_params', 'result_ticks', 'result_time'])
# deuruim = {
#     'BA': False,
#     'bipartite': False,
#     'lollipop': False,
#     'star': False,
#     'cubic': False
# }
# time_threshold = 3600
# for graphClass in ["BA", "lollipop"]:
#     for log_n in range(2,15):
#         n = 2**log_n
#         kStep = int(n/20)
#         if kStep == 0:
#             kStep = 1
#         for k in range(0,n,kStep):
#             filename = f'inputs/{graphClass}/{graphClass}_{n}.txt'
#             print(f'Começando {filename}_k:{k}.------------------------>')
#             if not deuruim[graphClass]:
#                 m2,v,e = ler_grafo(filename)
#                 status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
#                 if status != 101:
#                     deuruim[graphClass] = True
#                 results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
#                 print(f'------------------------>{filename}_k:{k} feito.')
#                 index += 1
#                 results.to_csv(f'./results7.csv')
# print("Fim.")


In [18]:
# index = 0
# results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
#                                  'extra_params', 'result_ticks', 'result_time'])
# deuruim = {
#     'BA': False,
#     'bipartite': False,
#     'lollipop': False,
#     'star': False,
#     'cubic': False
# }
# time_threshold = 3600
# for graphClass in ["cubic"]:
#     for log_n in range(2,15):
#         n = 2**log_n
#         kStep = int(n/20)
#         if kStep == 0:
#             kStep = 1
#         for k in range(0,n,kStep):
#             filename = f'inputs/{graphClass}/{graphClass}_{n}.txt'
#             print(f'Começando {filename}_k:{k}.------------------------>')
#             if not deuruim[graphClass]:
#                 m2,v,e = ler_grafo(filename)
#                 status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
#                 if status != 101:
#                     deuruim[graphClass] = True
#                 results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
#                 print(f'------------------------>{filename}_k:{k} feito.')
#                 index += 1
#                 results.to_csv(f'./results8.csv')

# print("Fim.")

Default row names c1, c2 ... being created.


Começando inputs/cubic/cubic_4.txt_k:0.------------------------>
4 vertices, 6 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 21 rows and 8 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)


Default row names c1, c2 ... being created.


------------------------>inputs/cubic/cubic_4.txt_k:0 feito.
Começando inputs/cubic/cubic_4.txt_k:1.------------------------>
4 vertices, 6 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 17 rows and 8 columns.
MIP Presolve modified 32 coefficients.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.03 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.03 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.03 ticks)


Default row names c1, c2 ... being created.


...

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 20481 rows, 8192 columns, and 57344 nonzeros.
Reduced MIP has 8192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.12 sec. (30.70 ticks)
Found incumbent of value 3654.000000 after 0.15 sec. (39.15 ticks)
Probing time = 0.02 sec. (3.64 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 20481 rows, 8192 columns, and 57344 nonzeros.
Reduced MIP has 8192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.28 sec. (127.78 ticks)
Probing time = 0.02 sec. (3.64 ticks)
Clique table members: 16384.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 2.40 sec. (2206.40 ticks)

        Nodes       

Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 20481 rows, 8192 columns, and 57344 nonzeros.
Reduced MIP has 8192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (30.70 ticks)
Found incumbent of value 4062.000000 after 0.06 sec. (39.17 ticks)
Probing time = 0.01 sec. (3.64 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 20481 rows, 8192 columns, and 57344 nonzeros.
Reduced MIP has 8192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (127.78 ticks)
Probing time = 0.01 sec. (3.64 ticks)
Clique table members: 16384.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 3.60 sec. (3374.26 ticks)

        Nodes       

Advanced basis not built.



Root node processing (before b&c):
  Real time             =   26.09 sec. (20240.75 ticks)
Sequential b&c:
  Real time             = 3573.91 sec. (3711611.65 ticks)
                          ------------
Total (root+branch&cut) = 3600.01 sec. (3731852.40 ticks)
------------------------>inputs/cubic/cubic_4096.txt_k:4080 feito.
Começando inputs/cubic/cubic_8192.txt_k:0.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:409.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:818.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:1227.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:1636.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:2045.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:2454.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:2863.------------------------>
Começando inputs/cubic/cubic_8192.txt_k:3272.------------------------>
Começando inputs/cubic/cubic_8192.tx

In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 3600
for graphClass in ["BA"]:
    for n in range(3, 512, 10):
        kStep = int(n/20)
        if kStep == 0:
            kStep = 1
        for k in range(0,n,kStep):
            filename = f'inputs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')
                index += 1
                results.to_csv(f'./results9.csv')
print("Fim.")


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 3600
for graphClass in ["BA", "lollipop"]:
    for log_n in range(2,15):
        n = 2**log_n
        kStep = int(n/20)
        if kStep == 0:
            kStep = 1
        for k in range(0,n,kStep):
            filename = f'inputs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')
                index += 1
                results.to_csv(f'./results10.csv')
print("Fim.")